# Librerias

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
#import pymongo

# MongoDB

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/") #connect

In [ ]:
channels_9now = client["channels_9now"] #db
tv_guide_data = channels_9now["tv_guide_data"] #collection
live_channels_now = channels_9now["live_channels_now"] #collection

# Info Canales

In [21]:
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Iniciar Selenium
driver = webdriver.Chrome()
driver.get("https://www.9now.com.au/live/channel-9")

# Medir tiempo de ejecución
start_time = time.time()

# Esperar que se carguen los canales
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "channel_card")))

# Encontrar los elementos con la clase "channel_card" y obtener los hrefs
channel_cards = driver.find_elements(By.CLASS_NAME, "channel_card")
hrefs = ["https://www.9now.com.au/live/channel-9"] + [
    card.get_attribute("href") for card in channel_cards if card.get_attribute("href")
]

descripcion = []
horario = []
categoria = []
info_prox_programa = []

# Iterar sobre cada link y extraer la descripción
for link in hrefs:
    driver.get(link)
    time.sleep(6)
    # Esperar a que la descripción esté presente
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@class='channel_card__metadata__description channel_card__metadata__description--collapsed']")))
    
    # Extraer la descripción
    description = driver.find_element(By.XPATH, "//*[@class='channel_card__metadata__description channel_card__metadata__description--collapsed']").text
    descripcion.append(description)
    horario.append(driver.find_element(By.XPATH, "(//*[@class='sc-be2dc1e9-1 eUEJCz sdui_inline_text_and_icon_element'])[1]").text)
    categoria.append(driver.find_element(By.XPATH, "(//*[@class='sc-be2dc1e9-1 eUEJCz sdui_inline_text_and_icon_element'])[2]").text)
    info_prox_programa.append(driver.find_element(By.XPATH, "(//*[@class='sc-cc1bfcf0-0 eqCRqz channel_card__metadata__element sdui_metadata__element_section'])[2]").text)

# Medir tiempo de ejecución
end_time = time.time()
execution_time = end_time - start_time
print(f"⏳ Tiempo total de ejecución: {execution_time:.2f} segundos")

# Crear el diccionario con los datos
live_channels_data = {
    "execution_time": execution_time,
    "channels": []
}

# Agregar los datos de los canales
for i in range(len(hrefs)):
    channel_data = {
        "link": hrefs[i],
        "description": descripcion[i],
        "schedule": horario[i],
        "category": categoria[i],
        "next_program_info": info_prox_programa[i]
    }
    live_channels_data["channels"].append(channel_data)

# Guardar los resultados en un archivo JSON
with open("live_channels_data.json", "w", encoding="utf-8") as json_file:
    json.dump(live_channels_data, json_file, ensure_ascii=False, indent=4)

live_channels_now.insert_one(channel_data)

# Cerrar el navegador
driver.quit()

print("Datos guardados en 'live_channels_data.json'")



KeyboardInterrupt



# TV Guide

In [ ]:
# Iniciar Selenium
driver = webdriver.Chrome()
driver.get("https://tvguide.9now.com.au/guide/")
driver.maximize_window()

# Medir tiempo de ejecución
start_time = time.time()

# Esperar que la lista de días esté disponible
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/main/div[4]/div[1]/div[4]/ul")))

# Encontrar los días en la barra de navegación
days = driver.find_elements(By.XPATH, "/html/body/div[1]/main/div[4]/div[1]/div[4]/ul/li")[0]
if days:
#for day in days:
    day_text = days[0].text
    programs_data = {}

    # Hacer clic en los dias
    day.click()
    time.sleep(2)

    # Esperar que se carguen los canales
    WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH, "//*[@class='js-channel-data']//div[contains(@class, 'guide__row') and @data-channel-id]")))
    channels = driver.find_elements(By.XPATH, "//*[@class='js-channel-data']//div[contains(@class, 'guide__row') and @data-channel-id]")

    for channel in channels:
        channel_name = channel.get_attribute("data-channel-name")
        programs_data[channel_name] = []

        # Obtener los programas (limitamos a 3 programas por canal)
        programas = channel.find_elements(By.XPATH, ".//*[contains(@class, 'guide__row__block')]//div//a[1]")

        for programa in programas:
            program_data = {
                "name": programa.find_element(By.XPATH, ".//h4").text,
                "link": programa.get_attribute("href"),
                "info": {
                    "time": "None",
                    "subtitles": "None",
                    "repeat": "None",
                    "description": "None",
                    "tags": "None"
                }
            }

            # Si el canal es "9Now", solo extraer el nombre y el link
            if channel_name == "9Now":
                try:
                    # Reintentar obtener el nombre del programa si ocurre StaleElementReferenceException
                    try:
                        program_data["name"] = programa.find_element(By.XPATH, ".//h4").text
                    except StaleElementReferenceException:
                        programas = channel.find_elements(By.XPATH, ".//*[contains(@class, 'guide__row__block')]//div//a[1]")
                        program_data["name"] = programas[programas.index(programa)].find_element(By.XPATH, ".//h4").text

                    # Agregar el programa al canal correspondiente
                    programs_data[channel_name].append(program_data)

                except Exception as e:
                    print(f"Error al extraer el programa de 9Now: {e}")
            else:
                try:
                    # Hacer clic en el programa para obtener más detalles
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(programa))
                    programa.click()
                    time.sleep(4)

                    # Extraer información del programa
                    try:
                        program_data["info"]["time"] = driver.find_element(By.XPATH, "//*[@class='show-down__timeFromTo']").text
                    except NoSuchElementException:
                        pass

                    try:
                        program_data["info"]["subtitles"] = driver.find_element(By.XPATH, "//*[@class='show-down__closedCaptions']").text
                    except NoSuchElementException:
                        pass

                    try:
                        program_data["info"]["repeat"] = driver.find_element(By.XPATH, "//*[@class='show-down__repeat']").text
                    except NoSuchElementException:
                        pass

                    try:
                        program_data["info"]["description"] = driver.find_element(By.XPATH, "//*[@class='show-down__description']").text
                    except NoSuchElementException:
                        pass

                    try:
                        program_data["info"]["tags"] = driver.find_element(By.XPATH, "//*[@class='show-down__tags']").text
                    except NoSuchElementException:
                        pass

                    programa.click()  # Cerrar la ventana del programa

                    # Agregar el programa al canal correspondiente
                    programs_data[channel_name].append(program_data)

                except Exception as e:
                    pass

            time.sleep(1)

     # Medir tiempo de ejecución
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"\n⏳ Tiempo total de ejecución: {execution_time:.2f} segundos")

    # Guardar los datos del día con el tiempo de ejecución
    final_data = {
        "day": day_text,
        "execution_time": execution_time,
        "programs": programs_data
    }

    # Guardar los resultados en un archivo JSON
    with open('programs_data.json', 'w', encoding='utf-8') as f:
        json.dump(final_data, f, ensure_ascii=False, indent=4)

    print("Datos guardados en 'programs_data.json'")


    # Insertar los datos en la colección de MongoDB
    #tv_guide_data.insert_one(final_data)

    print("Datos guardados en MongoDB")

# Cerrar Selenium
driver.quit()

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


WebDriverException: Message: target frame detached
  (failed to check if window was closed: disconnected: Unable to receive message from renderer)
  (Session info: chrome=133.0.6943.127)
Stacktrace:
	GetHandleVerifier [0x00007FF6CBAF6EE5+28773]
	(No symbol) [0x00007FF6CBA625D0]
	(No symbol) [0x00007FF6CB8F8DDC]
	(No symbol) [0x00007FF6CB8E60FB]
	(No symbol) [0x00007FF6CB8E4FAE]
	(No symbol) [0x00007FF6CB90973A]
	(No symbol) [0x00007FF6CB97EFDF]
	(No symbol) [0x00007FF6CB99EFE2]
	(No symbol) [0x00007FF6CB977023]
	(No symbol) [0x00007FF6CB93FF5E]
	(No symbol) [0x00007FF6CB9411E3]
	GetHandleVerifier [0x00007FF6CBE4422D+3490733]
	GetHandleVerifier [0x00007FF6CBE5BA13+3586963]
	GetHandleVerifier [0x00007FF6CBE5144D+3544525]
	GetHandleVerifier [0x00007FF6CBBBC9AA+838442]
	(No symbol) [0x00007FF6CBA6D01F]
	(No symbol) [0x00007FF6CBA695E4]
	(No symbol) [0x00007FF6CBA69786]
	(No symbol) [0x00007FF6CBA58CB9]
	BaseThreadInitThunk [0x00007FFD106F7374+20]
	RtlUserThreadStart [0x00007FFD1193CC91+33]